# import

In [1]:
import json
import random

# Recipe Recommendation code 

In [2]:
class RecipeRecommendation():

    def __init__(self, data_path):
        self.default_ingredients = {'물', ''}
        self.recipes = self._load_index(data_path)
        self.non_recipe = '적합한 레시피가 없습니다.\n 다시 시도해주세요.'

    def _load_index(self, data_path):
        '''
        요리 정보 json 파일 경로에서 데이터를 읽고 인덱싱

        :param data_path: 요리 정보 json 파일 경로
        :return: 인덱싱된 요리 리스트
        '''
        with open(data_path, "r", encoding='utf-8') as f:
            json_data = json.load(f)
        recipes = []
        for row in json_data['COOKRCP01']['row']:
            name = row["RCP_NM"]
            ingredient = row["RCP_PARTS_DTLS"]
            method = ""
            for i in range(1, 19):
                if i < 10:
                    mamula = row["MANUAL0" + str(i)]
                else:
                    mamula = row["MANUAL" + str(i)]
                method += mamula + "\n"

            method = method.strip()
            cooking_ingredients = ingredient.replace('\n', ', ')
            cooking_ingredients = [i.strip().split(' ')[0].split('(')[0] for i in cooking_ingredients.split(',')]

            cooking_ingredients = set(cooking_ingredients) - self.default_ingredients
            recipes.append({"dish": name, "ingredient": ingredient, "method": method, 'index': cooking_ingredients})
        return recipes

    def _find_recipe(self, find_ingredients):
        '''
        유사 요리 탐색
        요리 재료의 교집합이 가장 높은 조리법 중 랜덤으로 하나 선택
        :param find_ingredients: 사용자 검색 요리 재료
        :return: 추천 요리법
        '''
        find_ingredients = set(find_ingredients)
        result_ingredients = {}
        for recipe in self.recipes:
            recipe_ingredients = recipe['index']
            intersection = len(recipe_ingredients & find_ingredients)
            if intersection == 0:
                continue
            if intersection in result_ingredients:
                result_ingredients[intersection].append(recipe)
            else:
                result_ingredients[intersection] = [recipe]
        top_index = sorted(result_ingredients.keys(), reverse=True)[0]
        result_ingredient_list = result_ingredients[top_index]
        return random.choice(result_ingredient_list)

    def find_recipe(self, find_ingredients):
        '''
        추천 요리 반환
        :param find_ingredients: 사용자 검색 요리 재료
        :return: 추천 요리명과 요리법
        '''
        rec_recipe = self._find_recipe(find_ingredients)
        result = f"요리명 : {rec_recipe['dish']}\n=== 요리법 ===\n{rec_recipe['method']}"
        return result


# Recipe Recommendation 사용법
- 객체 선언

In [3]:
target_path = "./1000.json"
recipe_recommendation = RecipeRecommendation(target_path)

- 사용 예시 입니다.

In [15]:
find_ingredients = ['고구마죽', '', '']
recipe_recommendation = RecipeRecommendation(target_path)
result = recipe_recommendation.find_recipe(find_ingredients)
print(result)

요리명 : 칼륨 듬뿍 고구마죽
=== 요리법 ===
1. 고구마는 깨끗이 씻어서 껍질을 벗기고 4cm 정도로 잘라준다.a
2. 찜기에 고구마를 넣고 20~30분 정도 삶아 주고, 블렌더나 체를 이용하여 잘 으깨어 곱게 만든다.b
3. 고구마와 물을 섞어 끓이면서 찹쌀가루로 농도를 맞추고 설탕을 넣어 맛을 낸다.c
4. 잣을 팬에 노릇하게 볶아 다져서 고구마 죽에 섞는다. 기호에 따라 고구마를 튀겨 얹어 먹어도 좋다.
